In [100]:
import pandas as pd
import ast
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
import json
import numpy as np

In [101]:
movie_1 = pd.read_csv("final_movie_dataset.csv")

In [102]:
movie_1['release_date'] = pd.to_datetime(movie_1['release_date'], errors='coerce')

In [103]:
movie_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19942 entries, 0 to 19941
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   movieId            19942 non-null  float64       
 1   imdb_id            19942 non-null  object        
 2   original_title     19942 non-null  object        
 3   release_date       19942 non-null  datetime64[ns]
 4   runtime            19942 non-null  float64       
 5   title              19942 non-null  object        
 6   budget             19942 non-null  float64       
 7   revenue            19942 non-null  float64       
 8   popularity         19942 non-null  float64       
 9   vote_average       19942 non-null  float64       
 10  vote_count         19942 non-null  float64       
 11  director           19942 non-null  object        
 12  prod_comp_encoded  19942 non-null  float64       
 13  0                  19942 non-null  float64       
 14  1     

In [104]:
print(movie_1['release_date'].dt.year)

0        1995
1        1995
2        1995
3        1995
4        1995
         ... 
19937    1997
19938    2007
19939    2010
19940    2015
19941    2017
Name: release_date, Length: 19942, dtype: int32


In [105]:
cpi = pd.read_csv("cpi_data.csv")

In [106]:
print(cpi)

    Year         CPI
0   1947   22.331667
1   1948   24.045000
2   1949   23.809167
3   1950   24.062500
4   1951   25.973333
..   ...         ...
74  2021  270.967917
75  2022  292.625417
76  2023  304.704167
77  2024  313.697833
78  2025  319.492000

[79 rows x 2 columns]


In [107]:
# Iterating
for row in movie_1.itertuples():
    print(f"Movie Year: {row.release_date.year} {row.original_title}")

Movie Year: 1995 Toy Story
Movie Year: 1995 Jumanji
Movie Year: 1995 Grumpier Old Men
Movie Year: 1995 Waiting to Exhale
Movie Year: 1995 Father of the Bride Part II
Movie Year: 1995 Heat
Movie Year: 1995 Sabrina
Movie Year: 1995 Tom and Huck
Movie Year: 1995 Sudden Death
Movie Year: 1995 GoldenEye
Movie Year: 1995 The American President
Movie Year: 1995 Dracula: Dead and Loving It
Movie Year: 1995 Balto
Movie Year: 1995 Nixon
Movie Year: 1995 Cutthroat Island
Movie Year: 1995 Casino
Movie Year: 1995 Sense and Sensibility
Movie Year: 1995 Four Rooms
Movie Year: 1995 Ace Ventura: When Nature Calls
Movie Year: 1995 Money Train
Movie Year: 1995 Get Shorty
Movie Year: 1995 Copycat
Movie Year: 1995 Assassins
Movie Year: 1995 Powder
Movie Year: 1995 Leaving Las Vegas
Movie Year: 1995 Othello
Movie Year: 1995 Now and Then
Movie Year: 1995 Persuasion
Movie Year: 1995 La Cité des Enfants Perdus
Movie Year: 1995 摇啊摇，摇到外婆桥
Movie Year: 1995 Dangerous Minds
Movie Year: 1995 Twelve Monkeys
Movie Yea

In [108]:
movie_1.drop(movie_1.loc[movie_1['release_date'].dt.year < 1980].index, inplace=True)

print(movie_1)

        movieId    imdb_id               original_title release_date  runtime  \
0         862.0  tt0114709                    Toy Story   1995-10-30     81.0   
1        8844.0  tt0113497                      Jumanji   1995-12-15    104.0   
2       15602.0  tt0113228             Grumpier Old Men   1995-12-22    101.0   
3       31357.0  tt0114885            Waiting to Exhale   1995-12-22    127.0   
4       11862.0  tt0113041  Father of the Bride Part II   1995-02-10    106.0   
...         ...        ...                          ...          ...      ...   
19937   52103.0  tt0144020                      Xiao Wu   1997-01-01    105.0   
19938   63898.0  tt1110037                     Антидурь   2007-09-06     91.0   
19939   63266.0  tt1489253          Fobos. Klub Strakha   2010-03-25     79.0   
19940  327237.0  tt3814486       Bloed, Zweet en Tranen   2015-04-02    111.0   
19941  404604.0  tt5690142                          Maa   2017-07-07    146.0   

                           

In [109]:
movie_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18244 entries, 0 to 19941
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   movieId            18244 non-null  float64       
 1   imdb_id            18244 non-null  object        
 2   original_title     18244 non-null  object        
 3   release_date       18244 non-null  datetime64[ns]
 4   runtime            18244 non-null  float64       
 5   title              18244 non-null  object        
 6   budget             18244 non-null  float64       
 7   revenue            18244 non-null  float64       
 8   popularity         18244 non-null  float64       
 9   vote_average       18244 non-null  float64       
 10  vote_count         18244 non-null  float64       
 11  director           18244 non-null  object        
 12  prod_comp_encoded  18244 non-null  float64       
 13  0                  18244 non-null  float64       
 14  1          

In [110]:
def get_cpi(year):
    try:
        for row in cpi.itertuples():
            if(row.Year == year):
                # print('hi')
                # print(row.CPI)
                return row.CPI
    except:
        return None
    
cpi_2025 = get_cpi(2025)

inflation_values = []       
for row in movie_1.itertuples():
    cpi_release_year = get_cpi(row.release_date.year)

    if cpi_release_year is None:
        inflation_values.append(None)  # If no CPI, store None
    else:
        data = row.revenue * (cpi_2025 / cpi_release_year)
        inflation_values.append(data)

movie_1['inflation_adjusted_revenue'] = inflation_values


In [111]:
duplicates = movie_1.duplicated()

# Show all duplicated rows
print(movie_1[duplicates])

Empty DataFrame
Columns: [movieId, imdb_id, original_title, release_date, runtime, title, budget, revenue, popularity, vote_average, vote_count, director, prod_comp_encoded, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, genre_encoded, actor_encoded, inflation_adjusted_revenue]
Index: []

[0 rows x 30 columns]


In [112]:
movie_1.drop_duplicates(inplace=True)

In [113]:
movie_1.reset_index(drop=True, inplace=True)

In [114]:
movie_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18244 entries, 0 to 18243
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   movieId                     18244 non-null  float64       
 1   imdb_id                     18244 non-null  object        
 2   original_title              18244 non-null  object        
 3   release_date                18244 non-null  datetime64[ns]
 4   runtime                     18244 non-null  float64       
 5   title                       18244 non-null  object        
 6   budget                      18244 non-null  float64       
 7   revenue                     18244 non-null  float64       
 8   popularity                  18244 non-null  float64       
 9   vote_average                18244 non-null  float64       
 10  vote_count                  18244 non-null  float64       
 11  director                    18244 non-null  object    

In [115]:
movie_1.to_csv("final_movie_dataset_all_years.csv", index=False)

In [116]:
movie_filtered = movie_1.loc[movie_1['release_date'].dt.year >= 2000].copy()

In [117]:
duplicates_filtered = movie_filtered.duplicated()

# Show all duplicated rows
print(movie_filtered[duplicates_filtered])

Empty DataFrame
Columns: [movieId, imdb_id, original_title, release_date, runtime, title, budget, revenue, popularity, vote_average, vote_count, director, prod_comp_encoded, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, genre_encoded, actor_encoded, inflation_adjusted_revenue]
Index: []

[0 rows x 30 columns]


In [118]:
movie_filtered.drop_duplicates(inplace=True)

In [119]:
movie_filtered = movie_filtered.reset_index(drop=True)

In [120]:
movie_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13663 entries, 0 to 13662
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   movieId                     13663 non-null  float64       
 1   imdb_id                     13663 non-null  object        
 2   original_title              13663 non-null  object        
 3   release_date                13663 non-null  datetime64[ns]
 4   runtime                     13663 non-null  float64       
 5   title                       13663 non-null  object        
 6   budget                      13663 non-null  float64       
 7   revenue                     13663 non-null  float64       
 8   popularity                  13663 non-null  float64       
 9   vote_average                13663 non-null  float64       
 10  vote_count                  13663 non-null  float64       
 11  director                    13663 non-null  object    

In [121]:
movie_filtered.to_csv("final_movie_dataset_movies_greater_than_2000.csv", index=False)